## 실제 활용해보며 익히는 MySQL 기본 (중급) 문법
- 다양한 문법을 나열하기보다는, 실제 예제를 통해, 필요할 때, 그때그때 문법을 익혀야 이해가 빠름

### 1. Schema 정의
- TABLE 2개, FOREIGN KEY, PRIMARY KEY 사용

In [1]:
CREATE TABLE ranking ( 
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    math_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);

SyntaxError: invalid syntax (Temp/ipykernel_12456/1221797965.py, line 1)

In [ ]:
CREATE TABLE items ( 
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    discount_percent INT NOT NULL,
    provider VARCHAR(100)
);

#### 참고: 한글 처리에 문제가 있을 경우, DB, TABLE에 DEFAULT CHARSET=utf8 COLLATE=utf8_bin 옵션을 추가해서 실행
    CREATE DATABASE bestproducts DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
    CREATE TABLE tablename(feild definitions) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;

In [1]:
import pymysql
db = pymysql.connect(host= 'localhost', port = 3306, user = 'root', passwd='2031mysq!', db='bestproducts', charset='utf8')
cursor = db.cursor()

sql = '''
CREATE TABLE items ( 
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    discount_percent INT NOT NULL,
    provider VARCHAR(100)
);
''' 
cursor.execute(sql)
# FOREIGN KEY가 없는 것부터 생성해야 함

sql = '''
CREATE TABLE ranking ( 
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    math_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);
'''
cursor.execute(sql)

db.commit()
db.close()

OperationalError: (1050, "Table 'items' already exists")

### 2. 크롤링
- BeautifulSoup(bs4), Selenium, Scrapy(어렵지만 좋음)

#### main category 정보 가져오기

In [2]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    print ('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

http://corners.gmarket.co.kr//n/best ALL
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G01 패션의류
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G02 신발/잡화
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G03 화장품/헤어
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G04 유아동/출산
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G07 식품 
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G08 생활/주방/건강
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G09 가구/침구
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G05 스포츠/자동차
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G06 컴퓨터/전자
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G10 도서/음반
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G11 여행
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G12 e쿠폰


#### main/sub category 정보 가져오기

In [21]:
def get_category(category_link, category_name):
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    sub_categories = soup.select('div.navi.group ul li a')
    for sub_category in sub_categories:
        print(category_link, category_name, sub_category.get_text(), 'http://corners.gmarket.co.kr/' + sub_category['href'])

In [22]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

http://corners.gmarket.co.kr//n/best ALL
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G01 패션의류
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G02 신발/잡화
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G03 화장품/헤어
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G04 유아동/출산
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G07 식품 
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G08 생활/주방/건강
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G09 가구/침구
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G05 스포츠/자동차
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G06 컴퓨터/전자
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G10 도서/음반
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G11 여행
http://corners.gmarket.co.kr//n/best?viewType=G&groupCode=G12 e쿠폰


#### main/sub category + 상품 정보 + 상품 코드 + 판매자(제공자) 크롤링

In [19]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())

http://corners.gmarket.co.kr//n/best ALL


IndexError: list index out of range

In [18]:
def get_category(category_link, category_name):
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, 'ALL')
    
    sub_categories = soup.select('div.navi.group ul li > a')
    for sub_category in sub_categories:
        res = requests.get()
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_cateogory.get_text())

In [17]:
def get_items(html, category_name, sub_category_name):
    items_result_list = list()
    best_item = html.select('div.best-list')
    for index, item in enumerate(best_item[1].select('li')):
        ranking = index + 1
        title = item.select_one('a.itemname')
        ori_price = item.select_one('div.o-price')
        dis_price = item.select_one('div.s-price strong span')
        discount_percent = item.select_one('div.s-price em')
        # 없는 태그를 선택하는 경우 Null 값이 됨
        # 태그는 있는데 데이터가 없는 경우도 존재
        
        if ori_price == None or ori_price.get_text() == '':
            ori_price = dis_price
            
        if dis_price == None:
            ori_price, dis_price = 0, 0
        else :
            ori_price = ori_price.get_text().replace('', '').replace('원', '')
            dis_price = dis_price.get_text().replace('', '').replace('원', '')
            
        if disount_percent == None or discount_percent.get_text() == '':
            discount_percent = 0
        else :
            discount_percent.get_text().replace('%', '')
            
        product_link = item.select_one('div.thumb > a')
        item_code = product_link.attrs['href'].split('=')[1]
        
        res = requests.get(product_link.attrs['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > a > strong')
        if provider == None:
            provider = ''
        else :
            provider = provider.get_text()
            
        print(category_name, sub_category_name, title.get_text(), ori_price, dis_price, discount_percent)